In [191]:
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Kyle\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Kyle\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [192]:
import pandas as pd
import numpy as np

In [193]:
# read all data
ph = pd.read_csv('data/share-of-deaths-homicides.csv') # population/homicides
emis = pd.read_csv('data/GCB2022v27_MtCO2_flat.csv') # emissions
gdp = pd.read_csv('data/gdp.csv') # gdp
pop = pd.read_csv('data/population.csv') # population
ener = pd.read_csv('data/energy.csv') # energy


In [194]:
# Standardize the column names
ph.rename(columns={'Entity' : 'Country'}, inplace=True)
ph.rename(columns={'Deaths - Interpersonal violence - Sex: Both - Age: All Ages (Percent)' : 'Homicide Rate'}, inplace=True)

gdp.rename(columns={'Country Name' : 'Country'}, inplace=True)

pop.rename(columns={'Country Name' : 'Country'}, inplace=True) 

emis.rename(columns={'Total' : 'Total Emissions'}, inplace=True)
emis.rename(columns={'Per Capita' : 'Emissions Per Capita'}, inplace=True)

ener.rename(columns={'Energy_consumption' : 'Energy Consumption'}, inplace=True)
ener.rename(columns={'Energy_production' : 'Energy Production'}, inplace=True)

In [195]:
# drop unwanted columns
emis = emis.drop(columns = ['Other'])
emis = emis.drop(columns=['ISO 3166-1 alpha-3'])

ph = ph.drop(columns=['Code'])

gdp = gdp.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code'])

ener = ener.drop(columns=['GDP', 'Population', 'Energy_intensity_per_capita', 'Energy_intensity_by_GDP', 'CO2_emission'])

In [196]:
#debucketizing gdp
melted_gdp = gdp.melt(id_vars=['Country'], var_name="Year", value_name='GDP per capita')
#melted_gdp = melted_gdp.dropna()
melted_gdp = melted_gdp.sort_values(["Country","Year"])
melted_gdp = melted_gdp.reset_index(drop=True)
melted_gdp['Year'] = melted_gdp['Year'].astype(int)
melted_gdp['GDP per capita'] = melted_gdp['GDP per capita'].replace(np.nan,0)

#debucketizing population
melted_pop = pop.melt(id_vars=['Country'], var_name="Year", value_name='Population')
melted_pop = melted_pop.sort_values(["Country","Year"])
melted_pop = melted_pop.reset_index(drop=True)
melted_pop['Year'] = melted_pop['Year'].astype(int)
melted_pop['Population'] = melted_pop['Population'].replace(np.nan,0)



In [197]:
# drop rows with year < 1990 and year > 2019
emis = emis[emis['Year'] >= 1990]
emis = emis[emis['Year'] <= 2019]

gdp = melted_gdp[melted_gdp['Year'] >= 1990]
gdp = melted_gdp[melted_gdp['Year'] <= 2019]

pop = melted_pop[melted_pop['Year'] >= 1990]
pop = melted_pop[melted_pop['Year'] <= 2019]

ener = ener[ener['Year'] >= 1990]
ener = ener[ener['Year'] <= 2019]

In [198]:
# remove NaN values
emis.isnull().sum()

c = emis.groupby('Country')

c.apply(lambda x: x.isnull().sum())


,Country,Year,Total Emissions,Coal,Oil,Gas,Cement,Flaring,Emissions Per Capita
Country,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0
Albania,0,0,0,0,0,0,0,0,0
Algeria,0,0,0,0,0,0,0,0,0
Andorra,0,0,0,0,0,0,0,0,0
Angola,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
Viet Nam,0,0,0,0,0,0,0,0,0
Wallis and Futuna Islands,0,0,0,0,0,0,11,0,0
Yemen,0,0,0,0,0,0,0,0,0


In [199]:
# set year column as index
emis.set_index('Year', inplace=True)

# fill missing values with previous and forward 5-year averages
emis.fillna(emis.rolling(window=11, min_periods=1).mean().shift(1).fillna(method='bfill'), inplace=True)

# reset index
emis.reset_index(inplace=True)

emis.isnull().sum()

C:\Users\Kyle\AppData\Local\Temp\ipykernel_24752\1088784760.py:5: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Country'], dtype='object')
  emis.fillna(emis.rolling(window=11, min_periods=1).mean().shift(1).fillna(method='bfill'), inplace=True)


Year                    0
Country                 0
Total Emissions         0
Coal                    0
Oil                     0
Gas                     0
Cement                  0
Flaring                 0
Emissions Per Capita    0
dtype: int64

In [200]:
# replacing nan values with 0 and renaming energy types to match other tables
ener['Energy Consumption'] = ener['Energy Consumption'].replace(np.nan,0)
ener['Energy Production'] = ener['Energy Production'].replace(np.nan,0)

ener.replace('all_energy_types', 'Total', inplace=True)
ener.replace('coal', 'Coal', inplace=True)
ener.replace('petroleum_n_other_liquids', 'Oil', inplace=True)
ener.replace('natural_gas', 'Gas', inplace=True)
ener.replace('renewables_n_other', 'Other', inplace=True)
ener.replace('World', 'Global', inplace=True)

# pivot the energy table
ener_pivot = ener.pivot_table(index=['Country', 'Year'], columns='Energy_type', values=['Energy Consumption', 'Energy Production'])

# flatten the column names
ener_pivot.columns = [' '.join(col).strip() for col in ener_pivot.columns.values]

# drop nuclear energy
ener_pivot['Energy Production Total'] = ener_pivot['Energy Production Total'] - ener_pivot['Energy Production nuclear']
ener_pivot['Energy Consumption Total'] = ener_pivot['Energy Consumption Total'] - ener_pivot['Energy Consumption nuclear']
ener_pivot = ener_pivot.drop(columns=['Energy Consumption nuclear', 'Energy Production nuclear'])


ener_pivot[:10]

Energy Consumption Coal  Energy Consumption Gas  \
Country     Year                                                    
Afghanistan 1990                 0.002188                0.070151   
            1991                 0.001958                0.011517   
            1992                 0.000167                0.011092   
            1993                 0.000146                0.011092   
            1994                 0.000125                0.011088   
            1995                 0.000104                0.007392   
            1996                 0.000063                0.008504   
            1997                 0.000042                0.008504   
            1998                 0.000042                0.008504   
            1999                 0.000021                0.008504   

                  Energy Consumption Oil  Energy Consumption Other  \
Country     Year                                                     
Afghanistan 1990                0.028118                  0.007864   
            1991                0.026409                  0.007554   
            1992                0.014684                  0.005339   
            1993                0.014644                  0.005186   
            1994                0.013614                  0.005254   
            1995                0.013194                  0.004658   
            1996                0.012185                  0.004108   
            1997                0.011498                  0.003630   
            1998                0.011093                  0.003400   
            1999                0.011093                  0.003409   

                  Energy Consumption Total  Energy Production Coal  \
Country     Year                                                     
Afghanistan 1990                  0.108320                0.002078   
            1991                  0.047438                0.001860   
            1992                  0.031282                0.000158   
            1993                  0.031069                0.000139   
            1994                  0.030081                0.000119   
            1995                  0.025348                0.000099   
            1996                  0.024860                0.000059   
            1997                  0.023673                0.000040   
            1998                  0.023040                0.000040   
            1999                  0.023027                0.000020   

                  Energy Production Gas  Energy Production Oil  \
Country     Year                                                 
Afghanistan 1990               0.108891                    0.0   
            1991               0.011517                    0.0   
            1992               0.011092                    0.0   
            1993               0.011092                    0.0   
            1994               0.011088                    0.0   
            1995               0.007392                    0.0   
            1996               0.008504                    0.0   
            1997               0.008504                    0.0   
            1998               0.008504                    0.0   
            1999               0.008504                    0.0   

                  Energy Production Other  Energy Production Total  
Country     Year                                                    
Afghanistan 1990                 0.007864                 0.118833  
            1991                 0.007128                 0.020505  
            1992                 0.004892                 0.016143  
            1993                 0.004845                 0.016076  
            1994                 0.004818                 0.016024  
            1995                 0.004249                 0.011739  
            1996                 0.003733                 0.012296  
            1997                 0.003289                 0.011832  
            1998        

In [201]:
# merging tables
merged = pd.merge(ph, emis, how='inner', on=['Country', 'Year'])
merged = pd.merge(merged, gdp, how="left", on=['Country','Year'])
merged = pd.merge(merged, pop, how='left', on=['Country','Year'])
merged = pd.merge(merged, ener_pivot, how='left', on=['Country','Year'])


In [202]:
# checking for duplicates
duplicates = merged.duplicated(subset=['Country', 'Year'], keep=False)

# print resulting dataframe
# if there are no duplicates, the dataframe will be empty
print(merged[duplicates])

Empty DataFrame
Columns: [Country, Year, Homicide Rate, Total Emissions, Coal, Oil, Gas, Cement, Flaring, Emissions Per Capita, GDP per capita, Population, Energy Consumption Coal, Energy Consumption Gas, Energy Consumption Oil, Energy Consumption Other, Energy Consumption Total, Energy Production Coal, Energy Production Gas, Energy Production Oil, Energy Production Other, Energy Production Total]
Index: []

[0 rows x 22 columns]


In [203]:
merged.describe()

,Year,Homicide Rate,Total Emissions,Coal,Oil,Gas,Cement,Flaring,Emissions Per Capita,GDP per capita,...,Energy Consumption Coal,Energy Consumption Gas,Energy Consumption Oil,Energy Consumption Other,Energy Consumption Total,Energy Production Coal,Energy Production Gas,Energy Production Oil,Energy Production Other,Energy Production Total
count,5610.000000,5610.000000,5610.000000,5610.000000,5610.000000,5610.000000,5610.000000,5610.000000,5610.000000,4950.000000,...,5340.000000,5340.000000,5340.000000,5340.000000,5340.000000,5340.000000,5340.000000,5340.000000,5.340000e+03,5340.000000
mean,2004.500000,1.095742,120.147593,50.909146,39.571093,22.193680,5.057668,1.462085,4.545368,10250.963517,...,0.587916,0.435346,0.703165,0.178799,2.000478,0.575298,0.442260,0.800361,1.847518e-01,2.097923
std,8.656213,1.678670,543.209645,381.784589,105.091306,68.811459,37.316321,4.788171,6.223573,16763.887997,...,4.753741,1.366483,1.811879,0.727782,7.692726,4.549213,1.787962,2.581848,7.567735e-01,7.515651
min,1990.000000,0.040000,0.000000,0.000000,0.003664,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.163438,0.000000,0.000000,0.000000,0.000000,-1.000000e-39,0.000000
25%,1997.000000,0.260000,1.233389,0.000000,0.974387,0.000000,0.000097,0.000000,0.565035,801.488133,...,0.000000,0.000000,0.017163,0.000305,0.024634,0.000000,0.000000,0.000000,2.063384e-04,0.002375
50%,2004.500000,0.560000,9.157214,0.124576,4.934715,0.230832,0.400382,0.000000,2.376270,3040.517926,...,0.001177,0.003697,0.091011,0.013933,0.191635,0.000000,0.000000,0.000000,1.238702e-02,0.084069
75%,2012.000000,1.140000,58.731783,8.010227,28.476452,12.577810,1.877339,0.347518,6.495875,11253.254178,...,0.072162,0.244403,0.493736,0.087079,1.176062,0.009509,0.122577,0.150594,8.410134e-02,1.120101
max,2019.000000,17.740000,10740.996069,7543.157408,1559.728160,825.415088,826.876048,62.654678,66.817851,123678.702100,...,96.118714,17.779703,27.905786,17.642632,151.608886,88.513087,25.713639,25.671631,1.799732e+01,123.591360


In [204]:
#Generating measures
merged['Homicide per capita'] = (merged['Homicide Rate'] / merged['Population']) * 100000
merged['Homicide per emissions'] = merged['Homicide Rate'] / merged['Total Emissions']
merged['GDP'] = merged['GDP per capita'] * merged['Population']
merged['Energy Production to Consumption'] = merged['Energy Production Total'] / merged['Energy Consumption Total']

In [205]:
merged[:25]

,Country,Year,Homicide Rate,Total Emissions,Coal,Oil,Gas,Cement,Flaring,Emissions Per Capita,...,Energy Consumption Other,Energy Consumption Total,Energy Production Coal,Energy Production Gas,Energy Production Oil,Energy Production Other,Energy Production Total,Homicide per capita,Homicide per emissions,GDP
0,Afghanistan,1990,0.84,2.024326,0.278464,1.271408,0.403040,0.045766,0.025648,0.189281,...,0.007864,0.108320,0.002078,0.108891,0.0,0.007864,0.118833,0.007854,0.414953,0.000000e+00
1,Afghanistan,1991,1.04,1.914301,0.249627,1.204085,0.389125,0.045766,0.025697,0.178155,...,0.007554,0.047438,0.001860,0.011517,0.0,0.007128,0.020505,0.009679,0.543279,0.000000e+00
2,Afghanistan,1992,1.10,1.482054,0.021984,1.029584,0.362736,0.045766,0.021984,0.122916,...,0.005339,0.031282,0.000158,0.011092,0.0,0.004892,0.016143,0.009123,0.742213,0.000000e+00
3,Afghanistan,1993,1.15,1.486943,0.018320,1.047904,0.351744,0.046991,0.021984,0.106182,...,0.005186,0.031069,0.000139,0.011092,0.0,0.004845,0.016076,0.008212,0.773399,0.000000e+00
4,Afghanistan,1994,1.18,1.453829,0.014693,1.032171,0.337935,0.046991,0.022039,0.094065,...,0.005254,0.030081,0.000119,0.011088,0.0,0.004818,0.016024,0.007635,0.811650,0.000000e+00
5,Afghanistan,1995,1.20,1.417327,0.014656,1.011264,0.322432,0.046991,0.021984,0.086323,...,0.004658,0.025348,0.000099,0.007392,0.0,0.004249,0.011739,0.007309,0.846664,0.000000e+00
6,Afghanistan,1996,1.33,1.370104,0.007328,0.985616,0.307776,0.047400,0.021984,0.080092,...,0.004108,0.024860,0.000059,0.008504,0.0,0.003733,0.012296,0.007775,0.970729,0.000000e+00
7,Afghanistan,1997,1.17,1.304152,0.003664,0.948976,0.282128,0.047400,0.021984,0.073313,...,0.003630,0.023673,0.000040,0.008504,0.0,0.003289,0.011832,0.006577,0.897135,0.000000e+00
8,Afghanistan,1998,1.13,1.278504,0.003664,0.941648,0.263808,0.047400,0.021984,0.069134,...,0.003400,0.023040,0.000040,0.008504,0.0,0.003059,0.011603,0.006110,0.883845,0.000000e+00
9,Afghanistan,1999,1.13,1.091640,0.003664,0.776768,0.241824,0.047400,0.021984,0.056671,...,0.003409,0.023027,0.000020,0.008504,0.0,0.003068,0.011592,0.005866,1.035140,0.000000e+00


In [206]:
# replacing any missing values / infinities with 0
merged = merged.replace(np.nan, 0)
merged = merged.fillna(0)
merged['Homicide per emissions'].replace([np.inf, -np.inf], 0, inplace=True)

In [207]:
merged.sum()

Country                     AfghanistanAfghanistanAfghanistanAfghanistanAf...
Year                                                                 11245245
Homicide Rate                                                         6147.11
Total Emissions                                                  674027.99492
Coal                                                            285600.306574
Oil                                                             221993.830495
Gas                                                             124506.542015
Cement                                                             28373.5179
Flaring                                                           8202.295118
Emissions Per Capita                                             25499.513081
GDP per capita                                                50742269.408442
Population                                                     164767741468.0
Energy Consumption Coal                                         

In [208]:
merged.dtypes

Country                      object
Year                          int64
Homicide Rate               float64
Total Emissions             float64
Coal                        float64
Oil                         float64
Gas                         float64
Cement                      float64
Flaring                     float64
Emissions Per Capita        float64
GDP per capita              float64
Population                  float64
Energy Consumption Coal     float64
Energy Consumption Gas      float64
Energy Consumption Oil      float64
Energy Consumption Other    float64
Energy Consumption Total    float64
Energy Production Coal      float64
Energy Production Gas       float64
Energy Production Oil       float64
Energy Production Other     float64
Energy Production Total     float64
Homicide per capita         float64
Homicide per emissions      float64
GDP                         float64
dtype: object

In [209]:
merged.to_csv("~/Desktop/datascience/phase2/CSI4142-Project/out.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'C:\Users\Kyle\Desktop\datascience\phase2\CSI4142-Project'